In [5]:
import pandas as pd

# Define file paths (excluding unemployment_cost_index)
filepaths = {
    'unemployment': '../World_Datasets/Unemployment.csv',
    'current_account_balance': '../World_Datasets/Current_Account_Balance.csv',
    'exchange_rate_usd': '../World_Datasets/Exchange_Rate_USD.csv',
    'gdp': '../World_Datasets/GDP.csv',
    'government_debt': '../World_Datasets/Government_Debt.csv',
    'inflation': '../World_Datasets/Inflation.csv',
    'total_reserves': '../World_Datasets/Total_Reserves.csv'}

# Load all datasets into a dictionary of DataFrames
dataframes = {}
for name, filepath in filepaths.items():
    df = pd.read_csv(filepath)
    
    # Remove 'Series Name' and 'Series Code' columns if they exist
    df = df.drop(columns=['Series Name', 'Series Code'], errors='ignore')
    
    # Rename columns to include the dataset name and year
    df.columns = [
        col if col == 'Country Name' else f"{name}_{col.split(' ')[0]}" if '[' in col else f"{name}_{col}" 
        for col in df.columns]
    # Store the processed DataFrame
    dataframes[name] = df

# Identify the dataset with the least rows
least_rows_key = min(dataframes, key=lambda k: len(dataframes[k]))
print(f"Dataset with the least rows: {least_rows_key}")

# Use the dataset with the least rows as the base
merged_df = dataframes[least_rows_key]

# Perform inner joins with all other datasets on the 'Country Name' column
for name, df in dataframes.items():
    if name != least_rows_key:
        merged_df = pd.merge(merged_df, df, on=f'Country Name', how='inner')

# Keep only one 'Country Code' column
country_code_columns = [col for col in merged_df.columns if 'Country Code' in col]
if country_code_columns:
    merged_df = merged_df.drop(columns=country_code_columns[1:], errors='ignore')  # Drop duplicates, keep the first

# Save the merged dataset to a new CSV file
merged_df.to_csv('../World_Datasets/world_data.csv', index=False)

print("Merged dataset saved successfully as 'world_data.csv'")


Dataset with the least rows: unemployment
Merged dataset saved successfully as 'world_data.csv'
